In [96]:
import json
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense


In [97]:
#opening already translated and tagged dataset
with open('translated_dataframe.json', 'r', encoding='utf-8') as file:
    translated_data = json.load(file)

In [98]:
translated_df = pd.DataFrame(translated_data)

In [99]:
#need this bc too lasy to rewrite the json
translated_df['tag'][130] = str(2)
translated_df['tag'][157] = str(2)
translated_df['tag'][530] = str(4)
translated_df['tag'][653] = str(1)
translated_df['tag'][813] = str(1)

In [100]:
X = translated_df['original'].values
str_y = translated_df['tag'].values

In [101]:
#making tags int instead of str and fixing zero-based indexing
y = []
for i in str_y:
    i = int(i)-1
    y.append(i)

In [102]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [103]:
#finding the number of unique words - vocabulary size
def num_words(sentence):
  words_list = []
  for sent in sentence:
    for word in sent.split():
      if word.lower() not in words_list:
        words_list.append(word.lower())
      else:
        pass
  return words_list

evrth_list = []
for sentence in translated_df['translated']:
    evrth_list.append(sentence)

    
num_words = num_words(evrth_list)
len(num_words)

1620

In [104]:
#tokenization
max_words = 1620  # Define the maximum number of words to keep in the vocabulary
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

In [105]:
#finding the longest and the shortest sentence
count_list = []
for sentence in translated_df['translated']:
    splitted = sentence.split()
    count_list.append(len(splitted))

print(max(count_list))
print(min(count_list))

10
1


In [106]:
max_sequence_length = 10  # Define the maximum sequence length
X_train_pad = pad_sequences(X_train_seq, maxlen=max_sequence_length)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_sequence_length)

In [107]:
num_classes = len(np.unique(y))
y_train_encoded = to_categorical(y_train, num_classes)
y_test_encoded = to_categorical(y_test, num_classes)

In [108]:
model = Sequential()
model.add(Embedding(max_words, 128, input_length=max_sequence_length))
model.add(LSTM(128))
model.add(Dense(num_classes, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

batch_size = 64
epochs = 10
model.fit(X_train_pad, y_train_encoded, batch_size=batch_size, epochs=epochs, validation_data=(X_test_pad, y_test_encoded))


Epoch 1/10
12/12 [==============================] - 2s 54ms/step - loss: 1.5753 - accuracy: 0.2421 - val_loss: 1.5283 - val_accuracy: 0.2788
Epoch 2/10
12/12 [==============================] - 0s 18ms/step - loss: 1.5227 - accuracy: 0.3590 - val_loss: 1.5112 - val_accuracy: 0.3462
Epoch 3/10
12/12 [==============================] - 0s 20ms/step - loss: 1.4850 - accuracy: 0.3741 - val_loss: 1.4855 - val_accuracy: 0.2949
Epoch 4/10
12/12 [==============================] - 0s 20ms/step - loss: 1.4043 - accuracy: 0.5585 - val_loss: 1.4361 - val_accuracy: 0.3718
Epoch 5/10
12/12 [==============================] - 0s 20ms/step - loss: 1.2275 - accuracy: 0.5722 - val_loss: 1.3374 - val_accuracy: 0.5000
Epoch 6/10
12/12 [==============================] - 0s 21ms/step - loss: 0.9130 - accuracy: 0.7400 - val_loss: 1.2372 - val_accuracy: 0.4968
Epoch 7/10
12/12 [==============================] - 0s 20ms/step - loss: 0.5645 - accuracy: 0.8583 - val_loss: 1.3134 - val_accuracy: 0.5064
Epoch 8/10
12

In [109]:
new_texts = ['Помыть посуду']
new_texts_seq = tokenizer.texts_to_sequences(new_texts)
new_texts_pad = pad_sequences(new_texts_seq, maxlen=max_sequence_length)

predictions = model.predict(new_texts_pad)
print(predictions)

1/1 [==============================] - 0s 331ms/step
[[0.05608143 0.11524739 0.00988332 0.4392699  0.379518  ]]
